In [1]:
import numpy as np
train_x = np.load("data/single/train_x.npy")
train_y = np.load("data/single/train_y.npy")
test_x = np.load("data/single/test_x.npy")
test_y = np.load("data/single/test_y.npy")
import tensorflow as tf

In [2]:
train_y = np.apply_along_axis(lambda x: 1 if x >= 20 else 0, 1, train_y.reshape(60 * 8000, 1))
test_y = np.apply_along_axis(lambda x: 1 if x >= 20 else 0, 1, test_y.reshape(60 * 2000, 1))
train_y = np.repeat(train_y, 2)
test_y = np.repeat(test_y, 2)

for i in range(0, 60 * 8000):
    if (train_y[2 * i] == 0):
        train_y[2 * i] = 1
    else:
        train_y[2 * i] = 0
        train_y[2 * i + 1] = 1

for i in range(0, 60 * 2000):
    if (test_y[2 * i] == 0):
        test_y[2 * i] = 1
    else:
        train_y[2 * i] = 0
        train_y[2 * i + 1] = 1

In [3]:
train_y = train_y.reshape(60 * 8000, 2)
test_y = test_y.reshape(60 * 2000, 2)

In [4]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = tf.Session(config=config)

In [5]:
n_input = 101 * 101
x = tf.placeholder(tf.uint8, shape=[None, n_input])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W ,strides=[1, 1, 1 ,1], padding="SAME")
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1], padding="SAME")

# First Convolutional Layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 101, 101, 1])
float_x = tf.to_float(x_image)
h_conv1 = tf.nn.relu(conv2d(float_x, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Secondary Convolution Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Densely Connected Layer
W_fc1 = weight_variable([26 * 26 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 26 * 26 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Read Layer
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [6]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [7]:
from sklearn.utils import shuffle
train_x, train_y = shuffle(train_x, train_y, random_state = 0)
test_x, test_y = shuffle(test_x, test_y, random_state = 0)

In [8]:
sess.run(tf.global_variables_initializer())

In [ ]:
import sys
train_size = 40
test_size = 100
def get_batch_train(x, y, i, train_size):
    return x[i * train_size: (i + 1) * train_size], y[i * train_size: (i + 1) * train_size]

def get_batch_test(x, y, i, test_size):
    return x[i * test_size: (i + 1) * test_size], y[i * test_size: (i + 1) * test_size]

total_train_iteration = len(train_x) / train_size
for i in range(4000):
    for train_batch_step in range(int(total_train_iteration)):
        if train_batch_step % 400 == 0:
            str_log = ("%.2f %% " % (train_batch_step * 100 / total_train_iteration))
            sys.stdout.write('\r' + str_log)
        batch_x, batch_y = get_batch_train(train_x, train_y, train_batch_step, train_size)
        sess.run(train_step, feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5})
    
    total_iteration = len(test_x) / test_size
    total_accuracy = 0.0
    print("start evaluate")
    for test_batch_step in range(int(total_iteration)):
        test_batch_x, test_batch_y = get_batch_test(test_x, test_y, test_batch_step, test_size)
        total_accuracy += accuracy.eval(feed_dict={
            x:test_batch_x, y_: test_batch_y, keep_prob: 1.0}, session=sess)
    
    print("step %d, training accuracy %g"%(i, total_accuracy / total_iteration))

96.67 % start evaluate
step 0, training accuracy 0.702592
96.67 % start evaluate
step 1, training accuracy 0.627542
10.00 % 

In [38]:
y_result = tf.nn.softmax(y_conv)
test = sess.run(y_result, feed_dict = {x: train_x[110:130], keep_prob: 0.5})